In [1]:
%pip install llama-index
%pip install llama-index-readers-elasticsearch
%pip install llama-index-vector-stores-opensearch

%pip install llama-index-embeddings-ollama
%pip install ollama
%pip install nest-asyncio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from llama_index.core import SimpleDirectoryReader
reader = SimpleDirectoryReader(input_dir="pdfs",recursive=True)
documents = reader.load_data()

In [3]:
from llama_index.core.node_parser import TokenTextSplitter
splitter = TokenTextSplitter(
    chunk_size=512,
    chunk_overlap=128,
    separator=" ",
)
nodes = splitter.get_nodes_from_documents(
    documents, show_progress=True
)

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
nodes

[TextNode(id_='42acf4ef-cc38-4a96-b9e4-714246d9684a', embedding=None, metadata={'page_label': '1', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-27', 'last_modified_date': '2024-07-27'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e6b2c5bb-3f58-4e32-a348-71e03d93f450', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-27', 'l

In [5]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [6]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embedding_model_name = 'BAAI/bge-m3'
embed_model = HuggingFaceEmbedding(model_name=embedding_model_name,max_length=512, device=device)

In [7]:
embeddings = embed_model.get_text_embedding("box")
dim = len(embeddings)

In [8]:
dim

1024

In [9]:
! ! curl -XPUT "http://localhost:9200/_search/pipeline/hybrid-search-pipeline" -H 'Content-Type: application/json' -d' \
{ \
  "description": "Post processor for hybrid search", \
  "phase_results_processors": [ \
    { \
      "normalization-processor": { \
        "normalization": { \
          "technique": "min_max" \
        }, \
        "combination": { \
          "technique": "harmonic_mean", \
          "parameters": { \
            "weights": [ \
              0.3, \
              0.7 \
            ] \
          } \
        } \
      } \
    } \
  ] \
}'

{"acknowledged":true}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
import nest_asyncio
nest_asyncio.apply()

In [11]:
###dim=512

In [12]:
print(dim)

1024


In [13]:
from os import getenv
from llama_index.vector_stores.opensearch import (
    OpensearchVectorStore,
    OpensearchVectorClient,
)

# http endpoint for your cluster (opensearch required for vector index usage)
endpoint = getenv("OPENSEARCH_ENDPOINT", "http://localhost:9200")
# index to demonstrate the VectorStore impl
idx = getenv("OPENSEARCH_INDEX", "hybrid_pdf_index")

# OpensearchVectorClient stores text in this field by default
text_field = "content"
# OpensearchVectorClient stores embeddings in this field by default
embedding_field = "embedding"
# OpensearchVectorClient encapsulates logic for a
# single opensearch index with vector search enabled with hybrid search pipeline
client = OpensearchVectorClient(
    endpoint=endpoint,
    index=idx,
    dim=dim,
    embedding_field=embedding_field,
    text_field=text_field,
    search_pipeline="hybrid-search-pipeline",
)

# initialize vector store
vector_store = OpensearchVectorStore(client)

In [14]:
idx

'hybrid_pdf_index'

In [15]:
from llama_index.core import VectorStoreIndex, StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes, storage_context=storage_context, embed_model=embed_model
)

In [16]:
! curl -XGET http://localhost:9200/hybrid_pdf_index/_mapping | jq .

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1205  100  1205    0     0   381k      0 --:--:-- --:--:-- --:--:--  392k
{
  "hybrid_pdf_index": {
    "mappings": {
      "properties": {
        "content": {
          "type": "text",
          "fields": {
            "keyword": {
              "type": "keyword",
              "ignore_above": 256
            }
          }
        },
        "embedding": {
          "type": "knn_vector",
          "dimension": 1024,
          "method": {
            "engine": "nmslib",
            "space_type": "l2",
            "name": "hnsw",
            "parameters": {
              "ef_construction": 256,
              "m": 48
            }
          }
        },
        "metadata": {
          "properties": {
            "_node_content": {
              "type": "text",
              "fields": {
                "keyword": {
            

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode
retriever = index.as_retriever(similarity_top_k=3,vector_store_query_mode=VectorStoreQueryMode.HYBRID)
text_retriveve = "ที่มาของปัญหาจราจร"
prompt = retriever.retrieve(text_retriveve)
for r in prompt:
    print(r.metadata)
    print(r)

{'page_label': '2', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-27', 'last_modified_date': '2024-07-27'}
Node ID: 3d0c3a98-5a3a-4514-9c44-1a266dfe1dde
Text: ๒    ขออธิบายหัวข้อพื้นฐานที่เกี่ยวกับการจราจรเพื่อ
ช่วยเสริมให้เกิดความรู้ ความเข้าใจในตัวกฎหมายหลัก ได้ง่าย มากขึ้น
ดังนี้    ที่มาของปัญหา จราจรเกิดได้จากปัจจัยส าคัญ   ๔ ประการ คือ
๑. ถนนและผังเมือง   ๒. ปริมาณรถและทิศทางการเดินรถ   ๓.
พฤติกรรมการขับขี่   ๔. การบริหารงานจราจร    ถนนตามหลักวิศวกรรมจราจร
แบ่งเป็น  ๔ ประเภท   ๑. ทางด่วน  (Ex...
Score:  1.000



In [18]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode
retriever = index.as_retriever(similarity_top_k=3,vector_store_query_mode=VectorStoreQueryMode.HYBRID)
text_retriveve = "ผู้ขับขี่ คือใคร"
prompt = retriever.retrieve(text_retriveve)
for r in prompt:
    print(r.metadata)
    print(r)

{'page_label': '3', 'file_name': 'article_20160531140556.pdf', 'file_path': '/Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf', 'file_type': 'application/pdf', 'file_size': 301354, 'creation_date': '2024-07-27', 'last_modified_date': '2024-07-27'}
Node ID: 89cf0269-0789-4bb0-bc0c-cb1081f22ed8
Text: พ.ศ. ๒๕๒๒  “ผู้ขับขี่” หมายความว่า ผู้ขับรถ ผู้ประจ าเครื่อง
อุปกรณ์การขนส่งตามกฎหมายว่าด้วยการขนส่งตามกฎหมายว่าด้วยการขนส่ง
ผู้ลากเข็นยานพาหนะ  “คนเดินเท้า”   หมายความว่า
คนเดินและให้รวมตลอดถึงผู้ใช้เก้าอี้ล้อส าหรับคนพิการ  หรือรถส
าหรับเด็กด้วย    “เจ้าของรถ” หมายความรวมถึงผู้มีรถไว้ในครอบครองด้วย
“ผู้เก็บค่าโดยสาร ” หมายความว่าผู้ซึ่งรั...
Score:  1.000



In [19]:
from llama_index.vector_stores.opensearch import OpensearchVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core import Settings

# ตั้งค่า embed_model ที่คุณมีอยู่
Settings.embed_model = embed_model  # ใช้ embed_model ที่คุณโหลดมาแล้ว
Settings.llm = None  # ตั้งค่า LLM เป็น None เพื่อไม่ให้ใช้ OpenAI

# initialize vector store
vector_store = OpensearchVectorStore(client)

# สร้าง storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# สร้าง index โดยไม่ต้องใช้ nodes
index = VectorStoreIndex.from_vector_store(
    vector_store,
    storage_context=storage_context,
)

# สร้าง query engine
query_engine = index.as_query_engine()

# ค้นหา
response = query_engine.query("ผู้ขับขี่ คือใคร")

# แสดงผลลัพธ์
print(response)

LLM is explicitly disabled. Using MockLLM.
Context information is below.
---------------------
page_label: 3
file_path: /Users/grizzlystudio/RAG-with-Ollama/pdfs/article_20160531140556.pdf

พ.ศ. ๒๕๒๒  “ผู้ขับขี่” หมายความว่า ผู้ขับรถ ผู้ประจ าเครื่อง  
อุปกรณ์การขนส่งตามกฎหมายว่าด้วยการขนส่งตามกฎหมายว่าด้วยการขนส่ง  ผู้ลากเข็นยานพาหนะ  “คนเดินเท้า”  
หมายความว่า  คนเดินและให้รวมตลอดถึงผู้ใช้เก้าอี้ล้อส าหรับคนพิการ  หรือรถส าหรับเด็กด้วย   
“เจ้าของรถ” หมายความรวมถึงผู้มีรถไว้ในครอบครองด้วย  “ผู้เก็บค่าโดยสาร ” หมายความว่าผู้ซึ่งรับผิดชอบ
ในการเก็บค่าโดยสาร  และดูแลคนโดยสารที่อยู่ประจ ารถบรรทุกคนโดยสาร
---------------------
Given the context information and not prior knowledge, answer the query.
Query: ผู้ขับขี่ คือใคร
Answer: 
